In [13]:
from sklearn.metrics import accuracy_score
import shutil
import os, sys, time
import math
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from pathlib import Path
import numpy as np
from copy import deepcopy
from collections import defaultdict, OrderedDict

sys.path.append(os.getcwd()[:-3] + 'lib')
print(os.getcwd())
from datasets import timeSeriesDataset, FewShotSampler
from configs import get_parser, Logger, time_string, convert_secs2time, AverageMeter, obtain_accuracy
from models import euclidean_dist
import models
import datetime
from sklearn.neighbors import KNeighborsClassifier




def load_SFA_train_data(name):
    args_dataset_root = os.getcwd()[:-21]+'SFA_Python-master/test/BOSS_feature_Data_pyts/'
    train_dataset = timeSeriesDataset(args_dataset_root, 'train', name, 1, 10)
    return train_dataset


def random_choice(s_idxs,k):
    result=np.random.choice(s_idxs.size(0), k, replace=False)
    return s_idxs[result]
    


def get_log_path(name, ratio_number, ind_number):
    log_path = os.getcwd()[:-3] + 'logs/5shot_linear_transform/' + name + '/' + str(ratio_number) + '/' + str(
        ind_number)
    log_path = log_path + '/' + os.listdir(log_path)[0]
    log_path = log_path + '/baseline_classifier/model_linear_transform_lst.pth'
    return log_path



def save_to_file(sentence, dataset_name, log_path=None):
    father_path = './test_accuracy_log_all_as_proto/' + dataset_name
    if not os.path.exists(father_path):
        os.makedirs(father_path)
    path = father_path + '/' + dataset_name + 'NN_list_log.txt'
    if log_path != None:
        path = log_path
    with open(path, "a") as myfile:
        myfile.write(sentence + '\n')
    return path


name_list=[
    'ArrowHead',
    'BME',
    'CBF',
    'Chinatown',
    'ECG200',
    'GunPoint',
    'GunPointAgeSpan',
    'GunPointOldVersusYoung',
    'ItalyPowerDemand',
    'MoteStrain',
    'Plane',
    'SonyAIBORobotSurface1',
    'SonyAIBORobotSurface2',
    'SyntheticControl',
    'ToeSegmentation1',
    'TwoLeadECG',
    'UMD',
    'Wine',
  ]
ratio_number_list = [1]
ind_number_list = [10]
args_arch = 'linear_transform'
args_dataset_root = os.getcwd()[:-21]+'SFA_Python-master/test/BOSS_feature_Data_pyts/'
using_proto = True
pick_all =True

for name in name_list:
    for ind_radio, ratio_number in enumerate(ratio_number_list):
        for ind, ind_number in enumerate(ind_number_list):
                model_lst_path = get_log_path(name, ratio_number, ind_number)
                
                train_dataset = timeSeriesDataset(args_dataset_root, 'train', name, ratio_number, ind_number)
                model = models.__dict__[args_arch](train_dataset.fea_dim,256,64)
                model = torch.nn.DataParallel(model).cuda()

                checkpoint = torch.load(model_lst_path)
                start_epoch = checkpoint['epoch'] + 1
                best_acc = checkpoint['best_acc']
                model.load_state_dict(checkpoint['model_state_dict'])
                model.eval()
                
                
                feas = train_dataset.feature
                labels = train_dataset.label
                embs = model(feas)
                
                cpu_labels = labels.cpu().tolist()
                idxs_dict = defaultdict(list)
                for i, l in enumerate(cpu_labels):
                    idxs_dict[l].append(i)
                idxs_dict = dict(sorted(idxs_dict.items()))
                grouped_s_idxs = []
                for lab, idxs in idxs_dict.items():
                    print(lab)
                    grouped_s_idxs.append(torch.LongTensor(idxs[:]))
                proto_lst = [torch.mean(embs[s_idxs], dim=0) for s_idxs in grouped_s_idxs]
                proto = torch.stack(proto_lst, dim=0)
                logits = - euclidean_dist(embs, proto, transform=True).view(labels.size(0), len(proto))
                dis = logits.cpu().detach().numpy()
             
                print(dis)
                labind= 0
                result = []
                for s_idxs in grouped_s_idxs:
                    temp = dis[s_idxs]
                    print(labind)
                    result.append(s_idxs[np.argmax(temp,axis = 0)[labind]].numpy())
                    labind = labind + 1

                print(labels)
                print(result)
                sentence = ''
                for i in result:
                    sentence = sentence+str(i)+'\t'
                save_to_file(sentence,name)



/home/tangw/Desktop/old/DPSN/time_series_proto/exp
Building timeSeriesDataset for [ArrowHead] [train] with 3 classes ...
BOSS ACC IS 0.7885714285714286
0
1
2
[[-1.16269775e+03 -7.78053625e+05 -7.94485062e+05]
 [-1.00220650e+06 -1.83346699e+04 -3.34757750e+05]
 [-9.74954000e+05 -2.58456625e+05 -2.11454180e+04]
 [-1.38028652e+04 -9.13316375e+05 -9.59437625e+05]
 [-7.66539750e+05 -1.19805371e+03 -3.16253844e+05]
 [-5.60111062e+05 -2.52682422e+05 -1.39792266e+04]
 [-3.73568828e+04 -4.83773844e+05 -4.61155562e+05]
 [-4.39444188e+05 -4.17159609e+04 -2.64104500e+05]
 [-1.04297125e+06 -3.60926031e+05 -2.48187676e+04]
 [-1.54962168e+04 -9.49246938e+05 -8.47814688e+05]
 [-1.05902662e+06 -2.79266992e+04 -3.21917219e+05]
 [-7.35127438e+05 -2.47089953e+05 -1.28118030e+03]
 [-2.88984180e+03 -8.44492438e+05 -8.14735250e+05]
 [-1.06260150e+06 -2.74673496e+04 -3.83991250e+05]
 [-5.42631625e+05 -2.41080672e+05 -1.74464863e+04]
 [-2.64980737e+03 -7.30976812e+05 -7.80643000e+05]
 [-6.41139812e+05 -5.55688

[[-2.1641240e+04 -1.4321484e+05]
 [-6.7199219e+03 -1.9738041e+05]
 [-1.6016635e+04 -4.2166344e+05]
 [-4.2023987e+02 -2.6646094e+05]
 [-1.4366670e+04 -1.6459559e+05]
 [-5.2583301e+03 -3.5197619e+05]
 [-7.8727300e+03 -3.7272397e+05]
 [-2.4614497e+03 -2.2908375e+05]
 [-1.6035010e+04 -1.6102539e+05]
 [-2.6255635e+03 -3.2660131e+05]
 [-1.8653215e+04 -1.5209903e+05]
 [-9.6389209e+02 -2.8556662e+05]
 [-4.1182053e+02 -2.5736519e+05]
 [-1.7969479e+04 -4.3214856e+05]
 [-2.2318049e+04 -1.4121631e+05]
 [-1.1111412e+02 -2.7618938e+05]
 [-5.6356230e+03 -2.0552994e+05]
 [-2.1285901e+03 -2.2947880e+05]
 [-6.1441895e+03 -3.6107369e+05]
 [-5.8320625e+03 -2.0634453e+05]
 [-1.1028467e+04 -1.7778728e+05]
 [-6.7275574e+02 -2.9138669e+05]
 [-1.9881383e+04 -4.4201925e+05]
 [-3.0260364e+03 -3.3353981e+05]
 [-1.4236466e+03 -2.5449609e+05]
 [-1.5915876e+03 -2.4661311e+05]
 [-9.1107441e+03 -1.8500842e+05]
 [-6.7910632e+02 -2.9842688e+05]
 [-1.6971749e+03 -3.1573088e+05]
 [-2.8915754e+03 -2.2377630e+05]
 [-2.44341

[[-4.31017871e+03 -6.99957938e+05 -8.29024250e+05]
 [-1.13523936e+04 -5.14667125e+05 -7.43590438e+05]
 [-6.71366943e+03 -5.84443312e+05 -8.17388375e+05]
 [-3.41507080e+03 -5.98946375e+05 -7.18845688e+05]
 [-2.68056211e+04 -5.78996875e+05 -5.34444062e+05]
 [-8.99114531e+04 -1.09940225e+06 -1.39889200e+06]
 [-8.85751562e+03 -7.64592375e+05 -7.23871812e+05]
 [-7.81338086e+03 -6.44192250e+05 -7.39325500e+05]
 [-3.59360718e+03 -6.62600938e+05 -7.00627938e+05]
 [-3.19353281e+04 -4.24397750e+05 -6.17519438e+05]
 [-8.99114531e+04 -1.09940225e+06 -1.39889200e+06]
 [-1.81626074e+04 -7.39633250e+05 -6.37042875e+05]
 [-8.01409500e+05 -8.34584375e+03 -8.72033375e+05]
 [-4.91721812e+05 -2.13305078e+04 -7.38132000e+05]
 [-8.41481438e+05 -1.51255322e+04 -7.76057375e+05]
 [-7.55538375e+05 -8.30233672e+04 -1.30444288e+06]
 [-6.38870000e+05 -9.25237891e+03 -6.66340938e+05]
 [-7.53941375e+05 -4.30606445e+03 -8.03621500e+05]
 [-7.57974938e+05 -5.84658008e+03 -8.75885750e+05]
 [-6.71705438e+05 -1.13059268e+